In [ ]:
%matplotlib inline

from copy import deepcopy
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime

In [ ]:

# 1) Chargement de la machine de référence
ref_path = join(DATA_DIR, "Machine", "Tesla_model_3.json")
M_ref = load(ref_path)

print("=" * 80)
print("GÉNÉRATION DATASET DÉMAGNÉTISATION SÉVÈRE - TESLA MODEL 3")
print("=" * 80)

# 2) Analyse de la machine de référence
print("\n ANALYSE DE LA MACHINE DE RÉFÉRENCE")
print("=" * 45)

print(f"Type de machine: {type(M_ref).__name__}")
print(f"Nom: {M_ref.name}")

# Analyser les aimants
if hasattr(M_ref.rotor, 'hole'):
    print(" Machine à aimants détectée")
    
    magnet_count = 0
    for hole in M_ref.rotor.hole:
        for attr_name in dir(hole):
            if attr_name.startswith('magnet_'):
                magnet = getattr(hole, attr_name)
                if magnet is not None and hasattr(magnet, 'mat_type'):
                    if hasattr(magnet.mat_type, 'mag') and hasattr(magnet.mat_type.mag, 'Br'):
                        original_br = magnet.mat_type.mag.Br
                        print(f"   Aimant {magnet_count + 1}: Br = {original_br:.3f} T")
                        magnet_count += 1
                        break
    
    print(f"  Nombre total d'aimants: {magnet_count}")
else:
    print(" Cette machine n'a pas d'aimants - pas adaptée pour la démagnétisation")
    print("   Type de rotor:", type(M_ref.rotor).__name__)

# 3) Fonction de validation pour démagnétisation sévère
def validate_severe_demagnetization_machine(machine, name):
    """Validation spécifique pour machines avec démagnétisation sévère"""
    errors = []
    warnings = []
    
    # Vérification entrefer
    airgap = machine.stator.Rint - machine.rotor.Rext
    if airgap <= 0:
        errors.append(f"Entrefer négatif: {airgap*1000:.2f} mm")
    elif airgap < 0.3e-3:
        warnings.append(f"Entrefer très petit: {airgap*1000:.2f} mm")
    
    # Vérification bobinage
    if hasattr(machine.stator.winding, 'Ntcoil'):
        if machine.stator.winding.Ntcoil < 5:
            errors.append(f"Tours par bobine trop faibles: {machine.stator.winding.Ntcoil}")
    
    # Vérification démagnétisation sévère
    if hasattr(machine.rotor, 'hole'):
        for hole in machine.rotor.hole:
            for attr_name in dir(hole):
                if attr_name.startswith('magnet_'):
                    magnet = getattr(hole, attr_name)
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag') and hasattr(magnet.mat_type.mag, 'Br'):
                            br_value = magnet.mat_type.mag.Br
                            if br_value < 0.3:  # Br très faible
                                warnings.append(f"Br très faible: {br_value:.3f} T")
                            elif br_value > 1.5:  # Br trop élevé
                                warnings.append(f"Br élevé: {br_value:.3f} T")
    
    return errors, warnings

# 4) Génération des machines avec démagnétisation sévère
print("\n GÉNÉRATION DES MACHINES DÉMAGNÉTISATION SÉVÈRE")
print("=" * 55)

# Définir les niveaux de démagnétisation sévère (30% à 70%)
severe_demagnetization_levels = np.linspace(0.30, 0.70, 9)  # 9 niveaux de 30% à 70%

print(f" Niveaux de démagnétisation sévère: {severe_demagnetization_levels*100}%")
print(f" Nombre de machines: {len(severe_demagnetization_levels)}")

machines = []
names = []

for i, demag_level in enumerate(severe_demagnetization_levels):
    print(f"\n--- Génération machine {i+1:2d}/{len(severe_demagnetization_levels)} ---")
    
    # Copier la machine de référence
    M = deepcopy(M_ref)
    
    # Appliquer la démagnétisation sévère
    if hasattr(M.rotor, 'hole'):
        for hole in M.rotor.hole:
            for attr_name in dir(hole):
                if attr_name.startswith('magnet_'):
                    magnet = getattr(hole, attr_name)
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag') and hasattr(magnet.mat_type.mag, 'Br'):
                            original_br = magnet.mat_type.mag.Br
                            new_br = original_br * (1 - demag_level)
                            magnet.mat_type.mag.Br = new_br
                            
                            print(f"   Aimant: Br {original_br:.3f} T → {new_br:.3f} T (-{demag_level*100:.1f}%)")
                            print(f"   Réduction: {(original_br - new_br)/original_br*100:.1f}%")
                            break
    
    # Nom de la machine
    machine_name = f"Tesla_Model3_Defaut_Demagnetisation_Severe_{demag_level*100:.0f}_Pourcent"
    
    machines.append(M)
    names.append(machine_name)
    
    print(f" {machine_name}")

# 5) Validation des machines générées
print(f"\n VALIDATION DES MACHINES GÉNÉRÉES")
print("=" * 45)

valid_machines = []
valid_names = []

for Mi, nm in zip(machines, names):
    print(f"\n--- Validation de {nm} ---")
    
    # Validation spécifique
    errors, warnings = validate_severe_demagnetization_machine(Mi, nm)
    
    if errors:
        print(f" ERREURS critiques:")
        for error in errors:
            print(f"   - {error}")
        print(f"   → Machine rejetée")
        continue
    
    if warnings:
        print(f"  Avertissements:")
        for warning in warnings:
            print(f"   - {warning}")
    
    print(f" {nm}: Validation réussie")
    valid_machines.append(Mi)
    valid_names.append(nm)

# 6) Sauvegarde du dataset
print(f"\n SAUVEGARDE DU DATASET DÉMAGNÉTISATION SÉVÈRE")
print("=" * 55)

# Création du dossier
save_dir = "Tesla_Model3_Demagnetisation_Severe_30_70"
os.makedirs(save_dir, exist_ok=True)

print(f" Dossier créé: {os.path.abspath(save_dir)}/")

# Sauvegarde de la machine de référence
ref_path = os.path.join(save_dir, "Tesla_Model3_Reference.json")
M_ref.save(ref_path)
print(f" Machine de référence sauvegardée")

# Sauvegarde des machines avec démagnétisation sévère
if valid_machines:
    print(f"\n SAUVEGARDE DES MACHINES VALIDES")
    print("=" * 40)
    
    for Mi, nm in zip(valid_machines, valid_names):
        out_path = os.path.join(save_dir, nm + ".json")
        Mi.save(out_path)
        print(f" Sauvegardé: {nm}")
    
    # Rapport de sauvegarde
    print(f"\n RAPPORT DE SAUVEGARDE")
    print("=" * 30)
    print(f" Total machines sauvegardées: {len(valid_names) + 1}")
    print(f"   - Machine de référence: 1")
    print(f"   - Machines démagnétisation sévère: {len(valid_names)}")
    print(f"   - Plage de démagnétisation: 30% à 70%")
    
    # Vérification des fichiers créés
    print(f"\n VÉRIFICATION DES FICHIERS")
    print("=" * 30)
    
    total_files = 0
    total_size = 0
    for root, dirs, files in os.walk(save_dir):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                total_files += 1
                total_size += file_size
                print(f" {file} - {file_size:,} octets")
    
    print(f"\n SUCCÈS: {total_files} fichiers JSON créés")
    print(f" Taille totale: {total_size:,} octets ({total_size/1024:.1f} KB)")
    
    # Affichage du chemin complet
    print(f"\n EMPLACEMENT FINAL:")
    print(f"  {os.path.abspath(save_dir)}/")
    
    # Liste des machines générées
    print(f"\n LISTE DES MACHINES GÉNÉRÉES:")
    print("=" * 35)
    print(f"1. Tesla_Model3_Reference.json")
    for i, name in enumerate(valid_names, 2):
        print(f"{i:2d}. {name}.json")
    
else:
    print(" Aucune machine valide générée")

print("\n" + "=" * 80)
print(" GÉNÉRATION DU DATASET DÉMAGNÉTISATION SÉVÈRE TERMINÉE")
print("=" * 80)